In [ ]:
%load_ext autoreload
%autoreload 2

# dataset

> Create a dataset of storm matches

In [ ]:
#| default_exp dataset

In [ ]:
#| export
from pathlib import Path
import geopandas as gpd
import sqlite3
import datetime
import numpy as np
import gc


from tathu.io import spatialite
from tathu.constants import KM_PER_DEGREE

import exp4.core
from exp4.core import *
#from exp4.core import load_relato

gc.collect()

c:\Users\caioa\mambaforge\envs\fast-env\lib\site-packages\osgeo\osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0

## Loading PreVots data

In [ ]:
#| hide

# Old and new prevots reports
pth_relatos = Path(r"C:\Users\caioa\TESE\Experimento_3\relatos\granizo_2018-2023_novos.csv")
relatos = load_relato(pth_relatos)


# Brazilian municipalities data
pth_ibge = Path(r"C:\Users\caioa\TESE\Experimento_3\ibge")
shp_cities = gpd.read_file(pth_ibge / "BR_MUNICIPIOS_2022" / "BR_MUNICIPIOS_2022.shp")
shp_cities.set_index(["NM_MUN", "SIGLA_UF"], inplace=True)


# Dbs of tracked storms
pth_db = Path(r"C:\Users\caioa\TESE\Experimento_3\dbs_v2")
table_name = "systems"


c:\Users\caioa\TESE\exp4\exp4\core.py:40: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df_relatos = pd.read_csv(pth,
c:\Users\caioa\TESE\exp4\exp4\core.py:40: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_relatos = pd.read_csv(pth,


In [ ]:
#| hide
# Remove trailing spaces and before city names
relatos[["cidade ", "uf"]] = relatos[["cidade ", "uf"]].replace(r"^ +| +$", r"", regex=True) 
#relatos.head(4)
#relatos.tail(4)

## Iterating to build the Database


In [ ]:
#| hide

# Query for intersection by date and PreVots coords
intersec_query = f"""
    SELECT
        name, min, mean, std, count, event, relationships,
        strftime('%Y-%m-%d %H:%M:00', date_time) as date,
        ST_AsBinary(geom) as geom,
        ST_Area(ST_Intersection(ST_Buffer(MakePoint(?, ?), ?), geom)) as intersection
    FROM
        systems
    WHERE
        date BETWEEN ? AND ?
        AND ST_Intersects(
            ST_Buffer(MakePoint(?, ?), ?), 
            geom
        )
    ORDER BY
        intersection DESC
"""

#Query by name for relative systems
name_query = f"""
    SELECT
        name, min, mean, std, count, event, relationships,
        strftime('%Y-%m-%d %H:%M:00', date_time) as date,
        ST_AsBinary(geom) as geom
    FROM
        systems
    WHERE
        name = ? OR relationships = ? OR name = ? 
    ORDER BY
        date ASC
"""

#Query just by name of relative systems
justname_query = f"""
    SELECT
        name, min, mean, std, count, event, relationships,
        strftime('%Y-%m-%d %H:%M:00', date_time) as date,
        ST_AsBinary(geom) as geom
    FROM
        systems
    WHERE
        name = ?  
    ORDER BY
        date ASC
"""


In [ ]:
#| export
from sqlalchemy.orm import declarative_base
import geoalchemy2
from geoalchemy2 import load_spatialite, WKTElement
from sqlalchemy import create_engine, Column, Integer, Float, String, DateTime, ForeignKey
from sqlalchemy.event import listen
import pandas as pd
from geoalchemy2.shape import from_shape
from sqlalchemy.orm import sessionmaker, relationship
import os

os.environ['SPATIALITE_LIBRARY_PATH'] = "C:\\Users\\caioa\\mambaforge\\envs\\fast-env\\Library\\bin\\mod_spatialite.dll"


Base = declarative_base()

class Storm(Base):
    """
        Table to store unique storms and their evolving identifiers through splits and merges
    """
    __tablename__ = "storms"
    
    id = Column(Integer, primary_key=True)
    identifier = Column(String, index=True)


class StormEvent(Base):
    """
        Table to store each event, corresponding to the storm's physical data in each satellite scene
    """
    __tablename__ = "storm_events"

    id = Column(Integer, primary_key=True)
    storm_id = Column(Integer, ForeignKey("storms.id"))
    datetime = Column(DateTime)
    mean_bt = Column(Float)
    min_bt = Column(Float)
    std_dev_bt = Column(Float)
    count = Column(Integer)
    event_type = Column(String)    
    geometry = Column(geoalchemy2.Geometry(geometry_type = "POLYGON"))
    storm = relationship("Storm", backref="events")


class Intersection(Base):
    """
        Table to store the matches of storm polygons and hail reports
    """
    __tablename__ = "intersections"
    
    id = Column(Integer, primary_key=True)
    storm_event_id = Column(Integer, ForeignKey("storm_events.id"))
    intersection_time = Column(DateTime)
    intersection_geom = Column(geoalchemy2.Geometry(geometry_type = "POLYGON"))
    storm_event = relationship("StormEvent", backref="intersections")
    

#### Updating the database with new records

These steps aim to assure there is no overlapping between new and old records and assure integrity of the tables.

Procedure:   

1. Check latest stored "intersection";
2. Copy all intersections from the same storm;
3. Verify the oldest of them - this will be the starting point;
4. (Optionally) Remove the "storm" with their "storm_events" and "intersections";
5. GoTo the reports DataFrame in the position of the starting point and iterate from there. 


In [ ]:
#| hide

engine = create_engine('sqlite:///full_database_v5.db', echo=True)  # SQLite database file
listen(engine, "connect", load_spatialite)

#conn_w = engine.connect() ## Only for when creating the tables for the first time

In [ ]:
#| hide

## ONLY RUN THIS WHEN CREATING THE TABLE
#conn_w = engine.connect() ## Only for when creating the tables for the first time
#conn_w.close()

Base.metadata.create_all(engine)  # Create tables

2024-05-02 17:10:06,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-02 17:10:06,290 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("storms")
2024-05-02 17:10:06,291 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-02 17:10:06,292 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("storms")
2024-05-02 17:10:06,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-02 17:10:06,293 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("storm_events")
2024-05-02 17:10:06,293 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-02 17:10:06,294 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("storm_events")
2024-05-02 17:10:06,294 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-02 17:10:06,295 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("intersections")
2024-05-02 17:10:06,295 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-02 17:10:06,296 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("intersections")
2024-05-02 17:10:06,296 INFO sqlalchemy.engine

In [ ]:
#| hide

# ONLY RUN THIS IF UPDATING THE DATABASE
#Step 1:

from sqlalchemy import desc
from geoalchemy2.shape import to_shape



# Assuming 'session' is already set up
Session = sessionmaker(bind=engine)

# For checking the last event added to the database
with Session() as session:
    latest_intersection_event = session.query(Intersection).order_by(desc(Intersection.intersection_time)).first()

    storm = latest_intersection_event.storm_event.storm
    data = [{
        "name": storm.identifier,
        "Event ID": event.id,
        "DateTime": event.datetime,
        "Geometry": to_shape(event.geometry)
        } for event in storm.events]
        
# Storing in a geodataframe        
storm_events_gdf = gpd.GeoDataFrame(data, geometry="Geometry", crs="EPSG:4326")

2024-05-07 14:37:38,380 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-07 14:37:38,382 INFO sqlalchemy.engine.Engine SELECT intersections.id AS intersections_id, intersections.storm_event_id AS intersections_storm_event_id, intersections.intersection_time AS intersections_intersection_time, AsEWKB(intersections.intersection_geom) AS intersections_intersection_geom 
FROM intersections ORDER BY intersections.intersection_time DESC
 LIMIT ? OFFSET ?
2024-05-07 14:37:38,383 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (1, 0)
2024-05-07 14:37:38,773 INFO sqlalchemy.engine.Engine SELECT storm_events.id AS storm_events_id, storm_events.storm_id AS storm_events_storm_id, storm_events.datetime AS storm_events_datetime, storm_events.mean_bt AS storm_events_mean_bt, storm_events.min_bt AS storm_events_min_bt, storm_events.std_dev_bt AS storm_events_std_dev_bt, storm_events.count AS storm_events_count, storm_events.event_type AS storm_events_event_type, AsEWKB(storm_events.geometr

### Start iteration for inserting cases in the database

In [ ]:
#| hide

count_unmatched = 0
count_multipoly = 0

year = 0

with open('fulldb_v5_log.txt', 'w') as f:
    f.write(f"Creating Database \n")


for rel_num, relato in relatos_sub.iterrows():

    
    print(f"Processing Relato {rel_num} out of {total_rel} \n")
    with open('fulldb_v5_log.txt', 'a') as f:
        f.write(f"Processing Relato {rel_num} out of {total_rel} \n")
    
    # Year for the database search and connection
    if year != relato["date_time"].year:
        
        # Update year
        year = relato["date_time"].year
        
        # Path to the respective database and connect
        db_file = [db_name for db_name in pth_db.glob(f"{str(year)}*")][0]
        conn = connect_db(pth_db / db_file)

        print(f"Loading Database {db_file} \n")
        with open('fulldb_v5_log.txt', 'a') as f:
            f.write(f"Loading Database {db_file} \n")

    else:
        pass # Useful for tracked polygons in yearly files 

    
    # Get reports specs for query
    dt = relato["dt_min"] # Report's time uncertainty

    # Event window (datetime +- PreVots time uncertainty)
    date_start = datetime.datetime.strftime(relato["date_time"] - datetime.timedelta(minutes = int(dt)),
                           '%Y-%m-%d %H:%M:00')
    
    date_end = datetime.datetime.strftime(relato["date_time"] + datetime.timedelta(minutes = int(dt)),
                           '%Y-%m-%d %H:%M:00')


    # Buffer (10Km + event uncertainty) corrected to degree
    buffer_size = (relato["dx_km"] + 10) / KM_PER_DEGREE


    # Params for query-intersect
    params = (relato["lon"], relato["lat"], 
              buffer_size, 
              date_start, date_end, 
              relato["lon"], relato["lat"], 
              buffer_size)

    gdf_filtered = query2gdf(conn, intersec_query, params, "geom")

    if gdf_filtered.empty: # In case there is no match for the hail report
        with open('fulldb_v5_log.txt', 'a') as f:
            f.write(f"No matches! \n")
            count_unmatched += 1

        continue

    # Search for related storm polygons (same storm)
    best_match = gdf_filtered.iloc[0,:]

    intersec_geom = best_match["geom"].intersection(relato["buffer"])

    if intersec_geom.geom_type == 'MultiPolygon':
        #print("Matched with multipolygon. Likely sputious data \n")
        with open('fulldb_v5_log.txt', 'a') as f:
            f.write(f"Matched with multipolygon. Likely spurious data. \n")   
            count_multipoly += 1
        continue



    Session = sessionmaker(bind=engine)
    
    #session = Session()
    with Session() as session:
        # Load and insert data
        

        # Check if the unique storm already exists
        storm = session.query(Storm).filter_by(identifier=best_match["name"]).first() # First storm event is always spontaneous
        
        if not storm:
            storm = session.query(Storm).filter_by(identifier=best_match['relationships']).first() # In case the origin of the split is already in the db
    
        if not storm:
            storm = Storm(identifier=best_match['name'])
            session.add(storm)
            session.commit()



            # Check names of match
            name_params = (best_match["name"], best_match["name"], best_match["relationships"]) # Case same name, case split of the storm, case match is the split of the storm
        
            # Query related storms
            gdf_related = query2gdf(conn, name_query, name_params, "geom")




            for index, row in gdf_related.iterrows():
    
                # Insert storm event
                storm_event = StormEvent(
                    storm_id = storm.id,
                    datetime=pd.to_datetime(row['date'],  format='%Y-%m-%d %H:%M:00'),
                    mean_bt=row['mean'],
                    min_bt=row['min'],
                    std_dev_bt=row['std'],
                    count=row["count"],
                    event_type=row['event'],
                    geometry=WKTElement(row['geom'].wkt)
                    #geometry=from_shape(row['geom'], srid=4326)  # Ensure SRID matches your data
                )

                session.add(storm_event)
                session.commit()  # Commit to ensure 'storm_event' has an 'id' before using it in 'Intersection'


                # Intersections
                if row[:6].equals(best_match[:6]): 
                
                    intersection = Intersection(
                        storm_event_id = storm_event.id,
                        intersection_time = pd.to_datetime(row['date'],  format='%Y-%m-%d %H:%M:00'),
                        intersection_geom = WKTElement(intersec_geom.wkt)
                        )
                    session.add(intersection)
    
                # Commit the session to save all changes
                session.commit()

        else:


            for storm_event in storm.events:
                
                if storm_event.datetime == pd.to_datetime(best_match["date"],  format='%Y-%m-%d %H:%M:00'):
                    

                    intersection = Intersection(
                        storm_event_id = storm_event.id,
                        intersection_time = pd.to_datetime(best_match['date'],  format='%Y-%m-%d %H:%M:00'),
                        intersection_geom = WKTElement(intersec_geom.wkt)
                        )
                    session.add(intersection)   
                    session.commit()         

    session.close()
    gc.collect()

with open('fulldb_v5_log.txt', 'a') as f:
    f.write(f" {count_unmatched} \t UNMATCHED. \n {count_multipoly} \t UNMATCHED. \n")   
        

Processing Relato 7874 out of 13151 

Loading Database C:\Users\caioa\TESE\Experimento_3\dbs_v2\20210101_20211231_systems-db.sqlite 

Processing Relato 7875 out of 13151 

2024-05-06 11:36:15,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-06 11:36:15,479 INFO sqlalchemy.engine.Engine SELECT storms.id AS storms_id, storms.identifier AS storms_identifier 
FROM storms 
WHERE storms.identifier = ?
 LIMIT ? OFFSET ?
2024-05-06 11:36:15,479 INFO sqlalchemy.engine.Engine [generated in 0.00044s] ('779bef06-35d7-4c67-8625-5451e4819c42', 1, 0)
2024-05-06 11:36:15,482 INFO sqlalchemy.engine.Engine SELECT storm_events.id AS storm_events_id, storm_events.storm_id AS storm_events_storm_id, storm_events.datetime AS storm_events_datetime, storm_events.mean_bt AS storm_events_mean_bt, storm_events.min_bt AS storm_events_min_bt, storm_events.std_dev_bt AS storm_events_std_dev_bt, storm_events.count AS storm_events_count, storm_events.event_type AS storm_events_event_type, AsEWKB(storm_events

## Example Query 1: Retrieve all storm events (whole lifecycle) associated with the first storm in the database

In [ ]:
#| hide
import geopandas as gpd
from shapely import wkt
from geoalchemy2.shape import to_shape


# Assuming session is set up
first_storm = session.query(Storm).first()
if first_storm:
    # Dictionary to organize the output
    data = [{
        "name": first_storm.identifier,
        "datetime": event.datetime,
        "mean_bt": event.mean_bt,
        "geometry": to_shape(event.geometry),
        "event_type": event.event_type
    } for event in first_storm.events]

    # Store the results in a GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry="geometry", crs="EPSG:4326")
    #print(gdf) # Uncomment if working with scripts instead of notebook
else:
    print("No storms found in the database.")


gdf # Jupyter Notebook only


2024-05-07 14:50:20,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-07 14:50:20,165 INFO sqlalchemy.engine.Engine SELECT storms.id AS storms_id, storms.identifier AS storms_identifier 
FROM storms
 LIMIT ? OFFSET ?
2024-05-07 14:50:20,166 INFO sqlalchemy.engine.Engine [generated in 0.00054s] (1, 0)
2024-05-07 14:50:20,167 INFO sqlalchemy.engine.Engine SELECT storm_events.id AS storm_events_id, storm_events.storm_id AS storm_events_storm_id, storm_events.datetime AS storm_events_datetime, storm_events.mean_bt AS storm_events_mean_bt, storm_events.min_bt AS storm_events_min_bt, storm_events.std_dev_bt AS storm_events_std_dev_bt, storm_events.count AS storm_events_count, storm_events.event_type AS storm_events_event_type, AsEWKB(storm_events.geometry) AS storm_events_geometry 
FROM storm_events 
WHERE ? = storm_events.storm_id
2024-05-07 14:50:20,167 INFO sqlalchemy.engine.Engine [cached since 761.4s ago] (1,)


,name,datetime,mean_bt,geometry,event_type
0,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 17:45:00,231.896277,"POLYGON ((-48.70090 -26.84396, -48.61104 -26.8...",SPONTANEOUS_GENERATION
1,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 18:00:00,231.074087,"POLYGON ((-48.55713 -26.86192, -48.41337 -26.8...",CONTINUITY
2,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 18:15:00,230.789725,"POLYGON ((-48.46728 -27.05956, -48.57510 -27.0...",CONTINUITY
3,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 18:30:00,230.924504,"POLYGON ((-48.05396 -26.82599, -47.98208 -26.8...",CONTINUITY
4,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 18:45:00,230.933879,"POLYGON ((-47.76644 -26.86192, -47.64065 -26.8...",CONTINUITY
5,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 19:00:00,230.608942,"POLYGON ((-47.80238 -26.86192, -47.73050 -26.8...",CONTINUITY
6,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 19:15:00,230.790061,"POLYGON ((-47.62268 -26.84396, -47.53282 -26.8...",CONTINUITY
7,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 19:30:00,231.307595,"POLYGON ((-47.44297 -26.82599, -47.31718 -26.8...",CONTINUITY
8,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 19:45:00,231.368544,"POLYGON ((-47.33515 -26.82599, -47.31718 -26.8...",CONTINUITY


## Example Query 2: Retrieve all intersections with hailstorm reports of the first storm together with the specific storm event of intersection

In [ ]:
if first_storm:
    data = [{
        "name": first_storm.identifier,
        "datetime": event.datetime,
        "mean_bt": event.mean_bt,
        "event_type": event.event_type,
        "intersection_geometry": to_shape(intersection.intersection_geom)
    } for event in first_storm.events for intersection in event.intersections]
    
    intersec_gdf = gpd.GeoDataFrame(data, geometry="intersection_geometry", crs="EPSG:4326")
    #print(intersec_gdf) ## Uncomment if working with scripts instead of notebook
else:
    print("No storms found in the database.")

intersec_gdf # just for notebooks
#data

2024-05-07 14:50:55,398 INFO sqlalchemy.engine.Engine SELECT intersections.id AS intersections_id, intersections.storm_event_id AS intersections_storm_event_id, intersections.intersection_time AS intersections_intersection_time, AsEWKB(intersections.intersection_geom) AS intersections_intersection_geom 
FROM intersections 
WHERE ? = intersections.storm_event_id
2024-05-07 14:50:55,398 INFO sqlalchemy.engine.Engine [cached since 95.9s ago] (1,)
2024-05-07 14:50:55,407 INFO sqlalchemy.engine.Engine SELECT intersections.id AS intersections_id, intersections.storm_event_id AS intersections_storm_event_id, intersections.intersection_time AS intersections_intersection_time, AsEWKB(intersections.intersection_geom) AS intersections_intersection_geom 
FROM intersections 
WHERE ? = intersections.storm_event_id
2024-05-07 14:50:55,408 INFO sqlalchemy.engine.Engine [cached since 95.91s ago] (2,)
2024-05-07 14:50:55,416 INFO sqlalchemy.engine.Engine SELECT intersections.id AS intersections_id, inte

,name,datetime,mean_bt,event_type,intersection_geometry
0,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2018-06-05 17:45:00,231.896277,SPONTANEOUS_GENERATION,"POLYGON ((-48.55713 -27.11346, -48.55713 -27.1..."


## Example Query 3: Retrieve all "storm_events" of the storm's whole lifecycle based on an intersection with hail report

In [ ]:
first_intersection = session.query(Intersection).first() #Based on the first intersection
if first_intersection:
    storm = first_intersection.storm_event.storm
    data = [{
        "name": storm.identifier,
        "Event ID": event.id,
        "DateTime": event.datetime,
        "Geometry": to_shape(event.geometry)
    } for event in storm.events]
    
    gdf = gpd.GeoDataFrame(data, geometry="Geometry", crs="EPSG:4326")
    #print(gdf) ## Uncomment if working with scripts instead of notebook
else:
    print("No intersections found in the database.")
gdf

2024-05-07 14:51:14,797 INFO sqlalchemy.engine.Engine SELECT intersections.id AS intersections_id, intersections.storm_event_id AS intersections_storm_event_id, intersections.intersection_time AS intersections_intersection_time, AsEWKB(intersections.intersection_geom) AS intersections_intersection_geom 
FROM intersections
 LIMIT ? OFFSET ?
2024-05-07 14:51:14,797 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (1, 0)


,name,Event ID,DateTime,Geometry
0,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,1,2018-06-05 17:45:00,"POLYGON ((-48.70090 -26.84396, -48.61104 -26.8..."
1,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,2,2018-06-05 18:00:00,"POLYGON ((-48.55713 -26.86192, -48.41337 -26.8..."
2,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,3,2018-06-05 18:15:00,"POLYGON ((-48.46728 -27.05956, -48.57510 -27.0..."
3,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,4,2018-06-05 18:30:00,"POLYGON ((-48.05396 -26.82599, -47.98208 -26.8..."
4,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,5,2018-06-05 18:45:00,"POLYGON ((-47.76644 -26.86192, -47.64065 -26.8..."
5,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,6,2018-06-05 19:00:00,"POLYGON ((-47.80238 -26.86192, -47.73050 -26.8..."
6,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,7,2018-06-05 19:15:00,"POLYGON ((-47.62268 -26.84396, -47.53282 -26.8..."
7,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,8,2018-06-05 19:30:00,"POLYGON ((-47.44297 -26.82599, -47.31718 -26.8..."
8,a9b41e50-7ff7-40b8-bc0f-e488e357ae78,9,2018-06-05 19:45:00,"POLYGON ((-47.33515 -26.82599, -47.31718 -26.8..."


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()